## Import

In [1]:
import numpy as np

## LU разложение

In [23]:
def get_max_elem(matrix):
    matrix = np.abs(matrix)
    max_elem = matrix.max()
    if max_elem < 1e-12:
        return -1, -1
    max_elem_index = np.where(matrix == max_elem)
    return max_elem_index[0][0], max_elem_index[1][0]

In [3]:
def changes(matrix, idx, P, Q, sign):
    max_elem_index = get_max_elem(matrix.copy()[idx:, idx:])
    max_row, max_col = max_elem_index
    if max_row == -1:
        return matrix, P, Q, 0
    max_row += idx
    max_col += idx
    if max_row != idx:
        sign *= -1
        matrix[[idx, max_row]] = matrix[[max_row, idx]]
        P[[idx, max_row]] = P[[max_row, idx]]
    if max_col != idx:
        sign *= -1
        matrix[:, [max_col, idx]] = matrix[:, [idx, max_col]]
        Q[:, [max_col, idx]] = Q[:, [idx, max_col]]
    return matrix, P, Q, sign

In [4]:
def lu_factorization(matrix, sign=1):
    n = matrix.shape[0]
    last_idx = 0
    P = np.eye(n)
    Q = np.eye(n)
    for i in range(n):
        matrix, P, Q, sign = changes(matrix, i, P, Q, sign)
        if sign == 0:
            last_idx = i
            break
        for j in range(i + 1, n):
            k = matrix[j, i] / matrix[i, i]
            matrix[j, i:] -= k * matrix[i, i:]
            matrix[j, i] = k
        last_idx = i
    L = np.eye(n)
    U = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if j < i:
                L[i, j] = matrix[i, j]
            else:
                U[i, j] = matrix[i, j]
    return L, U, P, Q, sign, last_idx

In [24]:
# Тест LU == PAQ
arr = []
for i in range(1000):
    size = np.random.randint(2, 10)
    a = np.random.randint(-256, 256, (size, size)).astype(float)
    L, U, P, Q, sign, last_idx = lu_factorization(a.copy(), 1)
    LU = L @ U
    PA = P @ a @ Q
    arr.append(np.allclose(LU, PA))
print(np.all(arr))

True


## Определитель матрицы через LU разложение

In [6]:
def detA(matrix):
    L, U, _, _, sign, last_idx = lu_factorization(matrix.copy())
    if last_idx != matrix.shape[0] - 1:
        return 0
    det = sign
    for i in range(matrix.shape[0]):
        det *= L[i, i] * U[i, i]
    return det


def detLU(L, U, sign):
    det = sign
    for i in range(L.shape[0]):
        det *= L[i, i] * U[i, i]
    return det

In [7]:
# Тест detA
arr = []
for i in range(1000):
    size = np.random.randint(2, 10)
    a = np.random.randint(0, 256, (size, size)).astype(float)
    det_a = detA(a.copy())
    arr.append(np.allclose(det_a, np.linalg.det(a)))
print(np.all(arr))

True


## Нахождение системы линеых уравнений вида Ax = b через LU разложение

In [8]:
def solve_linear_system(A, b):
    L, U, P, Q, sign, last_idx = lu_factorization(A.copy())
    nA = A.shape[0]
    detA = detLU(L, U, sign)
    if detA == 0:
        return np.zeros(nA)
    y = np.zeros(nA)
    Pb = P @ b
    for i in range(nA):
        y[i] = Pb[i]
        for j in range(i):
            y[i] -= L[i, j] * y[j]
        y[i] /= L[i, i]
    x = np.zeros(nA)
    for i in range(nA - 1, -1, -1):
        x[i] = y[i]
        for j in range(i + 1, nA):
            x[i] -= U[i, j] * x[j]
        x[i] /= U[i, i]

    x = Q @ x
    return x

In [26]:
# Тест solve_linear_system
arr = []
for i in range(1000):
    size = np.random.randint(2, 10)
    a = np.random.randint(0, 256, (size, size)).astype(float)
    a[:, 2] = a[:, 1] + a[:, 0]
    a[:, 3] = a[:, 1] - a[:, 0]
    b = np.random.randint(0, 256, size).astype(float)
    b = a @ b
    x = solve_linear_system(a.copy(), b.copy())
    arr.append(np.allclose(x, np.linalg.solve(a, b)))
    print(a, end="\n\n\n\n")
    print(b, end="\n\n\n\n")
    print(x)
    break
print(np.all(arr))

[[  1.   1.   2. 120. 244.]
 [154. 166. 320. 151. 157.]
 [ 58.  46. 104.  18.  27.]
 [ 29.  15.  44. 131. 201.]
 [247. 106. 353. 136.  24.]]



[ 90. 188.  97. 125. 221.]



[0. 0. 0. 0. 0.]
False


## Обратная матрица

In [10]:
def inverse_matrix(matrix):
    n = matrix.shape[0]
    inv = np.zeros((n, n))
    for i in range(n):
        b = np.zeros(n)
        b[i] = 1
        inv[:, i] = solve_linear_system(matrix.copy(), b)
    return inv

In [11]:
# Проверка обратной матрицы
arr = []
for i in range(1000):
    size = np.random.randint(2, 10)
    a = np.random.randint(0, 256, (size, size)).astype(float)
    inv_a = inverse_matrix(a.copy())
    arr.append(np.allclose(inv_a, np.linalg.inv(a)))
print(np.all(arr))

True


## Число обусловленности

In [12]:
def norm_matrix(matrix):
    return np.max(np.sum(np.abs(matrix), axis=1))


def condition_num(matrix: np.ndarray) -> float:
    return norm_matrix(matrix) * norm_matrix(inverse_matrix(matrix))

In [13]:
# Проверка числа обусловленности
arr = []
for i in range(1000):
    size = np.random.randint(2, 10)
    a = np.random.randint(0, 256, (size, size)).astype(float)
    cond_num = condition_num(a.copy())
    arr.append(np.allclose(cond_num, np.linalg.cond(a, p=np.inf)))
print(np.all(arr))

True


## Ранг матрицы

In [14]:
def matrix_rank(matrix):
    L, U, _, _, sign, last_idx = lu_factorization(matrix.copy())
    return last_idx + 1

In [15]:
arr = []
for i in range(1000):
    size = np.random.randint(2, 10)
    a = np.random.randint(0, 256, (size, size)).astype(float)
    rank = matrix_rank(a.copy())
    arr.append(np.allclose(rank, np.linalg.matrix_rank(a)))
print(np.all(arr))

True


## QR разложение методом отражений

In [16]:
def QR_factirization(matrix):
    n, m = matrix.shape
    Q = np.empty((n, n))
    u = np.empty((n, n))
    u[:, 0] = matrix[:, 0]
    Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])
    for i in range(1, n):
        u[:, i] = matrix[:, i]
        for j in range(i):
            u[:, i] -= (matrix[:, i] @ Q[:, j]) * Q[:, j]
        Q[:, i] = u[:, i] / np.linalg.norm(u[:, i])

    R = np.zeros((n, m))
    for i in range(n):
        for j in range(i, m):
            R[i, j] = matrix[:, j] @ Q[:, i]
    return Q, R

In [17]:
def qr_solve_linear(matrix, b):
    Q, R = QR_factirization(matrix)
    y = Q.T @ b
    x = np.zeros(matrix.shape[1])
    for i in range(matrix.shape[1] - 1, -1, -1):
        x[i] = y[i]
        for j in range(i + 1, matrix.shape[1]):
            x[i] -= R[i, j] * x[j]
        x[i] /= R[i, i]
    return x

In [18]:
arr = []
for i in range(1000):
    size = np.random.randint(2, 10)
    a = np.random.randint(0, 256, (size, size)).astype(float)
    b = np.random.randint(0, 256, size).astype(float)
    x = qr_solve_linear(a.copy(), b.copy())
    x_linalg = np.linalg.solve(a, b)
    arr.append(np.allclose(x, x_linalg))
print(np.all(arr))

True


## Метод зейделя и метод Якоби

In [19]:
def seidel(matrix, b):
    epsilon = 10e-6
    n = matrix.shape[0]
    x = np.zeros((n, 1))
    while True:
        new_x = np.copy(x)
        s1 = 0
        s2 = 0
        for i in range(n):
            for j in range(i):
                s1 += matrix[i][j] * new_x[j]
            for j in range(i + 1, n):
                s2 += matrix[i][j] * x[j]
            new_x[i] = (b[i] - s1 - s2) / matrix[i][i]
        if np.linalg.norm(new_x - x) <= epsilon:
            break
        x = new_x
    return x

In [20]:
def getBc(A, b):
    n = A.shape[0]
    B = A.copy()
    for i in range(n):
        for j in range(n):
            if i != j:
                B[i, j] /= -A[i, i]
            else:
                B[i, j] = 0

    c = b.copy()
    for i in range(n):
        c[i] /= A[i, i]

    return B, c

#вычисление нормы мотрицы
def norm_matrix(matrix):
    return np.max(np.sum(np.abs(matrix), axis=1))

#||B|| <= q < 1
def get_q(B):
    normB = norm_matrix(B)
    if normB < 1:
        q = normB
    if normB >= 1:
        q = 0.99
    return q

def Jacobi(A, b):
    B, c = getBc(A, b)
    q = get_q(B)
    cnt = 1
    n = B.shape[0]
    xnew = c.copy()
    norm = norm_matrix(c)

    while (q / (1 - q)) * norm > 1e-10:
        x = xnew.copy()
        xnew = np.dot(B, xnew) + c
        diff = xnew - x
        norm = norm_matrix(diff)
        cnt += 1

    return xnew, cnt

def Seidel(A, b):
    B, c = getBc(A, b)
    q = get_q(B)
    cnt = 1
    n = B.shape[0]
    xnew = c.copy()
    norm = norm_matrix(c)

    while (q / (1 - q)) * norm > 1e-10:
        x = xnew.copy()
        for i in range(n):
            xnew[i] = np.dot(B[i, :], xnew[:]) + c[i]
        diff = xnew - x
        norm = norm_matrix(diff)
        cnt += 1

    return xnew, cnt



def check_solve(A, x, b):
    return np.allclose(A @ x, b)

def is_pos(x):
    return np.all(np.linalg.eigvals(x) > 0)


In [21]:
def change_to_diag(R):
    A = R.copy()
    n = A.shape[0]
    for i in range(n):
        row_sum = np.sum(np.abs(A[i,:])) - np.abs(A[i,i])
        if np.abs(A[i,i]) <= row_sum:
            A[i,i] = row_sum + n**2
    return A

def symmetric_matrix(n):
    A = np.random.rand(n, n)
    return (A + A.T) / 2
 
def diag(A):
    for i in range (A.shape[0]):
      sum = 0
      for j in range (A.shape[0]):
          if (i != j):
            sum += abs(A[i][j])
      if (abs(A[i][i]) <= sum):
          return False 
      return True

In [22]:
from numpy import math


n = 3
A1 = np.random.randint(-100, 100, (n, n))
A1 = A1.astype(np.float64)
b = np.random.randint(-100, 100, (n, 1))
b = b.astype(np.float64)
A = change_to_diag(A1)

print('Рандомная матрица A с диагональным преобладанием размером', n, ' * ', n, ': ')
print(A)

print('Рандомный вектор-столбец b размером', n, ' * ', 1, ': ')
print(b)

B, c = getBc(A, b)
q = get_q(B) 

xt = np.linalg.solve(A, b)
print("Верное решение :", xt.T)

print("Метод Якоби:")
x, counterJ = Jacobi(A, b)


print('x = ', x.T)
print(check_solve(A, x, b))


print("Метод Зейделя:")
x, counterS = Seidel(A, b)


print('x = ', x.T)
print(check_solve(A, x, b))


k = math.ceil(math.log(((1e-10 / norm_matrix(c)) * (1 - q)), q))

print("Априорная оценка числа необходимых итераций: ", k)
print("Апостериорная оценка числа итераций Якоби: ", counterJ)
print("Апостериорная оценка числа итераций Зейделя: ", counterS)

print('\n\n\n')


A1 = np.array([[0.48760983, 0.45581607, 0.35726152],
 [0.45581607, 0.73184001, 0.54538553],
 [0.35726152, 0.54538553, 0.42117855]])

if (is_pos(A1) and diag(A1) == False):
    print('Положительно определенная матрица А без диагонального преобладания размером', n, ' * ', n, ': ')
    print(A1)
    B1, c1 = getBc(A1, b)
    q1 = get_q(B1) 

    xt1 = np.linalg.solve(A1, b)
    print("Верное решение :", xt1.T)

    print("Метод Якоби:")
    x1, counterJ1 = Jacobi(A1, b)
    print('x = ', x1.T)
    print(check_solve(A1, x1, b))

    print("Метод Зейделя:")
    x1, counterS1 = Seidel(A1, b)
    print('x = ', x1.T)
    print(check_solve(A1, x1, b))

    k1 = math.ceil(math.log(((1e-10 / norm_matrix(c1)) * (1 - q1)), q1))
    print("Априорная оценка числа необходимых итераций: ", k1)
    print("Апостериорная оценка числа итераций Якоби: ", counterJ1)
    print("Апостериорная оценка числа итераций Зейделя: ", counterS1)


Рандомная матрица A с диагональным преобладанием размером 3  *  3 : 
[[-83. -27. -40.]
 [-82. 121. -30.]
 [ 30. -55.  87.]]
Рандомный вектор-столбец b размером 3  *  1 : 
[[-80.]
 [-38.]
 [-35.]]
Верное решение : [[ 1.14937962  0.31646193 -0.59857451]]
Метод Якоби:
x =  [[ 1.14937962  0.31646193 -0.59857451]]
True
Метод Зейделя:
x =  [[ 1.14937962  0.31646193 -0.59857451]]
True
Априорная оценка числа необходимых итераций:  1151
Апостериорная оценка числа итераций Якоби:  56
Апостериорная оценка числа итераций Зейделя:  21




Положительно определенная матрица А без диагонального преобладания размером 3  *  3 : 
[[0.48760983 0.45581607 0.35726152]
 [0.45581607 0.73184001 0.54538553]
 [0.35726152 0.54538553 0.42117855]]
Верное решение : [[-264.65052744  215.51582002 -137.68450464]]
Метод Якоби:
x =  [[inf inf nan]]
False
Метод Зейделя:
x =  [[-264.65052744  215.51582002 -137.68450464]]
True
Априорная оценка числа необходимых итераций:  3257
Апостериорная оценка числа итераций Якоби:  134

/var/folders/8q/8qmry__52c7c1rb1p1rzhy7w0000gn/T/ipykernel_99188/1652722903.py:38: RuntimeWarning: overflow encountered in scalar multiply
  while (q / (1 - q)) * norm > 1e-10:
/var/folders/8q/8qmry__52c7c1rb1p1rzhy7w0000gn/T/ipykernel_99188/1652722903.py:41: RuntimeWarning: overflow encountered in subtract
  diff = xnew - x
